- competition/dataset : [https://www.kaggle.com/c/home-credit-default-risk/data](https://www.kaggle.com/c/home-credit-default-risk/data)
- date : 2021/02/02
- original : [https://www.kaggle.com/eliotbarr/stacking-test-sklearn-xgboost-catboost-lightgbm](https://www.kaggle.com/eliotbarr/stacking-test-sklearn-xgboost-catboost-lightgbm)

## Stacking Test-Sklearn, XGBoost, CatBoost, LightGBM

**✏ 필사 1회** 

[Allstate Faron의 스크립트](https://www.kaggle.com/mmueller/allstate-claims-severity/stacking-starter/run/390867)를 참고했습니다.  
전처리는 [ogrellier의 스크립트](https://www.kaggle.com/ogrellier/good-fun-with-ligthgbm)를 참고했습니다.

In [76]:
import pandas as pd
import numpy as np

from scipy.stats import skew
import xgboost as xgb

# from sklearn.cross_validation import KFold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from math import sqrt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import gc

import warnings
warnings.filterwarnings('ignore')

In [7]:
NFOLDS = 3
SEED = 0
NROWS = None

In [9]:
data = pd.read_csv('data/application_train.csv')
test = pd.read_csv('data/application_test.csv')
prev = pd.read_csv('data/previous_application.csv')

In [35]:
data.head(3)

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,1,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
test.head(3)

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,0,1,0,0,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,0,0,0,0,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,0,0,1,0,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0


In [37]:
prev.head(3)

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,0,1730.430,17145.0,17145.0,0.0,17145.0,0,15,...,0,12.0,0,0,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,1,25188.615,607500.0,679671.0,NaN,607500.0,1,11,...,1,36.0,1,1,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,1,15060.735,112500.0,136444.5,NaN,112500.0,2,11,...,1,12.0,2,2,365243.0,-271.0,59.0,365243.0,365243.0,1.0


In [15]:
print('%-30s | %7s | %10s | %10s | %10s'%('FEATURES', 'TYPE', 'NB VALUES', 'NBNaNS', 'NaNs (%)'))
for f_ in data:
    print('%-30s | %7s | %10s | %10s |  %5.2f'%(f_, str(data[f_].dtype),
                                                str(len(data[f_].value_counts(dropna=False))),
                                                str(data[f_].isnull().sum()),
                                                100*data[f_].isnull().sum()/data.shape[0]))

FEATURES                       |    TYPE |  NB VALUES |     NBNaNS |   NaNs (%)
SK_ID_CURR                     |   int64 |     307511 |          0 |   0.00
TARGET                         |   int64 |          2 |          0 |   0.00
NAME_CONTRACT_TYPE             |  object |          2 |          0 |   0.00
CODE_GENDER                    |  object |          3 |          0 |   0.00
FLAG_OWN_CAR                   |  object |          2 |          0 |   0.00
FLAG_OWN_REALTY                |  object |          2 |          0 |   0.00
CNT_CHILDREN                   |   int64 |         15 |          0 |   0.00
AMT_INCOME_TOTAL               | float64 |       2548 |          0 |   0.00
AMT_CREDIT                     | float64 |       5603 |          0 |   0.00
AMT_ANNUITY                    | float64 |      13673 |         12 |   0.00
AMT_GOODS_PRICE                | float64 |       1003 |        278 |   0.09
NAME_TYPE_SUITE                |  object |          8 |       1292 |   0.42
NAME_INC

In [16]:
categorical_feats = [f for f in data.columns if data[f].dtype == 'object']

for f_ in categorical_feats:
    data[f_], indexer = pd.factorize(data[f_])
    test[f_] = indexer.get_indexer(test[f_])

❗ indexer.get_indexer 부분 잘 이해 안됨

In [27]:
gc.enable()

y_train = data['TARGET']
del data['TARGET']

In [28]:
prev_cat_features = [f_ for f_ in prev.columns if prev[f_].dtype == 'object']

for f_ in prev_cat_features:
    prev[f_], _ = pd.factorize(prev[f_])

In [30]:
avg_prev = prev.groupby('SK_ID_CURR').mean()
cnt_prev = prev[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
avg_prev['np_app'] = cnt_prev['SK_ID_PREV']
del avg_prev['SK_ID_PREV']

In [38]:
avg_prev

,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,...,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,np_app
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,0.000000,3951.000000,24835.500,23787.00,2520.00,24835.500,4.00,13.000000,0.0,1.0,...,8.00,2.00,0.000000,365243.0,-1709.000000,-1499.000000,-1619.000000,-1612.000000,0.000000,1
100002,0.000000,9251.775000,179055.000,179055.00,0.00,179055.000,0.00,9.000000,0.0,1.0,...,24.00,3.00,8.000000,365243.0,-565.000000,125.000000,-25.000000,-17.000000,0.000000,1
100003,0.333333,56553.990000,435436.500,484191.00,3442.50,435436.500,3.00,14.666667,0.0,1.0,...,10.00,1.00,6.666667,365243.0,-1274.333333,-1004.333333,-1054.333333,-1047.333333,0.666667,3
100004,0.000000,5357.250000,24282.000,20106.00,4860.00,24282.000,4.00,5.000000,0.0,1.0,...,4.00,0.00,10.000000,365243.0,-784.000000,-694.000000,-724.000000,-714.000000,0.000000,1
100005,0.500000,4813.200000,22308.750,20076.75,4464.00,44617.500,2.50,10.500000,0.0,1.0,...,12.00,3.00,2.500000,365243.0,-706.000000,-376.000000,-466.000000,-460.000000,0.000000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456251,0.000000,6605.910000,40455.000,40455.00,0.00,40455.000,1.00,17.000000,0.0,1.0,...,8.00,2.00,0.000000,365243.0,-210.000000,0.000000,-30.000000,-25.000000,0.000000,1
456252,0.000000,10074.465000,57595.500,56821.50,3456.00,57595.500,6.00,10.000000,0.0,1.0,...,6.00,3.00,6.000000,365243.0,-2466.000000,-2316.000000,-2316.000000,-2311.000000,1.000000,1
456253,0.000000,4770.405000,24162.750,20625.75,4403.25,24162.750,1.00,11.500000,0.0,1.0,...,5.00,2.00,0.000000,365243.0,-2339.000000,-2219.000000,-2219.000000,-2212.500000,0.500000,2


In [50]:
x_train = data.merge(right=avg_prev.reset_index(), how='left', on='SK_ID_CURR')
x_test = test.merge(right=avg_prev.reset_index(), how='left', on='SK_ID_CURR')

x_train.fillna(0, inplace=True)
x_test.fillna(0, inplace=True)

ntrain = x_train.shape[0]
ntest = x_test.shape[0]

In [51]:
excluded_feats = ['SK_ID_CURR']
features = [f_ for f_ in x_train.columns if f_ not in excluded_feats]

x_train = x_train[features]
x_test = x_test[features]

In [52]:
kf = KFold(n_splits=NFOLDS, shuffle=True, random_state=SEED)

In [54]:
class SklearnWrapper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)
        
    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)
    
    def predict(self, x):
        return self.clf.predict_proba(x)[:, 1]

In [55]:
class CatboostWrapper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_seed'] = seed
        self.clf = clf(**params)
    
    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)
    
    def predict(self, x):
        return self.clf.predict_proba(x)[:, 1]

In [56]:
class LightGBMWrapper(object):
    def __init__(self, clf, seed=0, params=False):
        params['random_state'] = seed
        self.clf = clf(**params)
    
    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)
    
    def predict(self, x):
        return self.clf.pred_proba(x)[:, 1]

In [57]:
class XgbWrapper(object):
    def __init__(self, seed=0, params=None):
        self.param = params
        self.param['seed'] = seed
        self.nrounds = params.pop('nrounds', 250)
    
    def train(self, x_train, y_train):
        dtrain = xgb.DMatrix(x_train, label=y_train)
        self.gbdt = xgb.train(self.param, dtrain, self.nrounds)
    
    def predict(self, x):
        return self.gbdt.predict(xgb.DMatrix(x))

In [58]:
def get_oof(clf):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))
    
    for i, (train_index, test_index) in enumerate(kf.split(x_train)):
        x_tr = x_train.loc[train_index]
        y_tr = y_train.loc[train_index]
        x_te = x_train.loc[test_index]
        
        clf.train(x_tr, y_tr)
        
        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)
    
    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [64]:
et_params = {
    'n_jobs':16,
    'n_estimators':200,
    'max_features':0.5,
    'max_depth':12,
    'min_samples_leaf':2,
}

rf_params = {
    'n_jobs':16,
    'n_estimators':200,
    'max_features':0.2,
    'max_depth':12,
    'min_samples_leaf':2,
}

xgb_params = {
    'seed':0,
    'colsample_bytree':0.7,
    'silent':1,
    'subsample':0.7,
    'learning_rate':0.075,
    'objective':'binary:logistic',
    'max_depth':4,
    'num_parallel_tree':1,
    'min_child_weight':1,
    'nrounds':200
}

catboost_params = {
    'iterations':200,
    'learning_rate':0.5,
    'depth':3,
    'l2_leaf_reg':40,
    'bootstrap_type':'Bernoulli',
    'subsample':0.7,
    'scale_pos_weight':5,
    'eval_metric':'AUC',
    'od_type':'Iter',
    'allow_writing_files':False
}

lightgbm_params = {
    'n_estimators':200,
    'learning_rate':0.1,
    'num_leaves':123,
    'colsample_bytree':0.8,
    'subsample':0.9,
    'max_depth':15,
    'reg_alpha':0.1,
    'reg_lambda':0.1,
    'min_split_gain':0.01,
    'min_child_weight':2
}

In [67]:
xg = XgbWrapper(seed=SEED, params=xgb_params)
et = SklearnWrapper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
rf = SklearnWrapper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
cb = CatboostWrapper(clf=CatBoostClassifier, seed=SEED, params=catboost_params)
lg = LightGBMWrapper(clf=LGBMClassifier, seed=SEED, params=lightgbm_params)

In [69]:
xg_oof_train, xg_oof_test = get_oof(xg)

[12:43:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:43:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:44:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip

In [70]:
et_oof_train, et_oof_test = get_oof(et)

In [71]:
rf_oof_train, rf_oof_test = get_oof(rf)

In [72]:
cb_oof_train, cb_oof_test = get_oof(cb)

0:	total: 124ms	remaining: 24.8s
1:	total: 182ms	remaining: 18s
2:	total: 223ms	remaining: 14.7s
3:	total: 275ms	remaining: 13.5s
4:	total: 324ms	remaining: 12.6s
5:	total: 387ms	remaining: 12.5s
6:	total: 439ms	remaining: 12.1s
7:	total: 482ms	remaining: 11.6s
8:	total: 527ms	remaining: 11.2s
9:	total: 582ms	remaining: 11.1s
10:	total: 635ms	remaining: 10.9s
11:	total: 704ms	remaining: 11s
12:	total: 778ms	remaining: 11.2s
13:	total: 827ms	remaining: 11s
14:	total: 876ms	remaining: 10.8s
15:	total: 925ms	remaining: 10.6s
16:	total: 970ms	remaining: 10.4s
17:	total: 1.03s	remaining: 10.4s
18:	total: 1.07s	remaining: 10.2s
19:	total: 1.13s	remaining: 10.2s
20:	total: 1.19s	remaining: 10.1s
21:	total: 1.24s	remaining: 10s
22:	total: 1.28s	remaining: 9.87s
23:	total: 1.33s	remaining: 9.77s
24:	total: 1.38s	remaining: 9.66s
25:	total: 1.45s	remaining: 9.68s
26:	total: 1.5s	remaining: 9.59s
27:	total: 1.55s	remaining: 9.53s
28:	total: 1.63s	remaining: 9.61s
29:	total: 1.69s	remaining: 9.56s

In [73]:
print('XG-CV: {}'.format(sqrt(mean_squared_error(y_train, xg_oof_train))))
print('ET-CV: {}'.format(sqrt(mean_squared_error(y_train, et_oof_train))))
print('RF-CV: {}'.format(sqrt(mean_squared_error(y_train, rf_oof_train))))
print('CB-CV: {}'.format(sqrt(mean_squared_error(y_train, cb_oof_train))))

XG-CV: 0.25947405731861983
ET-CV: 0.26296502347137407
RF-CV: 0.2629856213006646
CB-CV: 0.33101423487560705


In [74]:
x_train = np.concatenate((xg_oof_train, et_oof_train, rf_oof_train, cb_oof_train), axis=1)
x_test = np.concatenate((xg_oof_test, et_oof_test, rf_oof_test, cb_oof_test), axis=1)

print('{}, {}'.format(x_train.shape, x_test.shape))

(307511, 4), (48744, 4)


In [77]:
logistic_regression = LogisticRegression()
logistic_regression.fit(x_train, y_train)

test['TARGET'] = logistic_regression.predict_proba(x_test)[:, 1]
test[['SK_ID_CURR', 'TARGET']].to_csv('data/submission_3.csv', index=False, float_format='%.8f')